In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
!pip install optuna-integration[catboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import optuna
import catboost as cb
import matplotlib
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from optuna.integration import CatBoostPruningCallback




In [4]:
df_train=pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
df_test=pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')



In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


In [6]:
df_train['store'].unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [7]:
df_train['product'].unique()

array(['Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler',
       'Kerneler Dark Mode'], dtype=object)

In [8]:
df_train['Day']=pd.to_datetime(df_train['date']).dt.strftime("%d")
df_train['Month']=pd.to_datetime(df_train['date']).dt.strftime("%m")
df_train['Year']=pd.to_datetime(df_train['date']).dt.strftime("%Y")
df_train['Day_of_week']=pd.to_datetime(df_train['date']).dt.dayofweek


In [9]:
df_train['store']=df_train['store'].astype('category')
df_train['country']=df_train['country'].astype('category')
df_train['product']=df_train['product'].astype('category')
df_train['Day']=df_train['Day'].astype('category')
df_train['Month']=df_train['Month'].astype('category')
df_train['Year']=df_train['Year'].astype('category')
df_train['Day_of_week']=df_train['Day_of_week'].astype('category')

In [10]:
df_test['Day']=pd.to_datetime(df_test['date']).dt.strftime("%d")
df_test['Month']=pd.to_datetime(df_test['date']).dt.strftime("%m")
df_test['Year']=pd.to_datetime(df_test['date']).dt.strftime("%Y")
df_test['Day_of_week']=pd.to_datetime(df_test['date']).dt.dayofweek

In [11]:
df_test['store']=df_test['store'].astype('category')
df_test['country']=df_test['country'].astype('category')
df_test['product']=df_test['product'].astype('category')
df_test['Day']=df_test['Day'].astype('category')
df_test['Month']=df_test['Month'].astype('category')
df_test['Year']=df_test['Year'].astype('category')
df_test['Day_of_week']=df_test['Day_of_week'].astype('category')

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   id           230130 non-null  int64   
 1   date         230130 non-null  object  
 2   country      230130 non-null  category
 3   store        230130 non-null  category
 4   product      230130 non-null  category
 5   num_sold     221259 non-null  float64 
 6   Day          230130 non-null  category
 7   Month        230130 non-null  category
 8   Year         230130 non-null  category
 9   Day_of_week  230130 non-null  category
dtypes: category(7), float64(1), int64(1), object(1)
memory usage: 6.8+ MB


In [13]:
df_train.set_index('date').head()

,id,country,store,product,num_sold,Day,Month,Year,Day_of_week
date,,,,,,,,,
2010-01-01,0,Canada,Discount Stickers,Holographic Goose,NaN,01,01,2010,4
2010-01-01,1,Canada,Discount Stickers,Kaggle,973.0,01,01,2010,4
2010-01-01,2,Canada,Discount Stickers,Kaggle Tiers,906.0,01,01,2010,4
2010-01-01,3,Canada,Discount Stickers,Kerneler,423.0,01,01,2010,4
2010-01-01,4,Canada,Discount Stickers,Kerneler Dark Mode,491.0,01,01,2010,4


In [14]:
temp=df_train.set_index('date').groupby(['country', 'store', 'product'])['num_sold']
temp.head()

<ipython-input-14-9352d877f2f4>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp=df_train.set_index('date').groupby(['country', 'store', 'product'])['num_sold']


date
2010-01-01       NaN
2010-01-01     973.0
2010-01-01     906.0
2010-01-01     423.0
2010-01-01     491.0
               ...  
2010-01-05     264.0
2010-01-05    1536.0
2010-01-05    1410.0
2010-01-05     770.0
2010-01-05     962.0
Name: num_sold, Length: 450, dtype: float64

In [15]:

df_train['date']=pd.to_datetime(df_train['date'])
df = df_train.set_index('date')
df = df.sort_values(by=['country', 'store', 'product', 'date'])

# Impute NaN with interpolation (grouped by Country, Store, and Product)
df['num_sold'] = df.groupby(['country', 'store', 'product'])['num_sold'].transform(
    lambda group: group.interpolate(method='linear',limit=None,limit_direction='both').ffill().bfill()
)
df['num_sold'].isna().any()

<ipython-input-15-eb72dc58358d>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['num_sold'] = df.groupby(['country', 'store', 'product'])['num_sold'].transform(


True

In [16]:
condition = (df.num_sold.notna()) & (df['country']=='Canada') & (df['store']=='Discount Stickers') & (df['product']=='Holographic Goose')
df[condition]


,id,country,store,product,num_sold,Day,Month,Year,Day_of_week
date,,,,,,,,,


In [17]:
#Get num_sold NaN product store and country
t=df[df.num_sold.isna()].groupby(['country','store','product'])['product'].value_counts().to_dict()

<ipython-input-17-ce1350fd27b3>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  t=df[df.num_sold.isna()].groupby(['country','store','product'])['product'].value_counts().to_dict()


In [18]:
#Country store and product that have NaN
for  k,v in t.items():
    if v!=0:
        print(k,v)

('Canada', 'Discount Stickers', 'Holographic Goose') 2557
('Kenya', 'Discount Stickers', 'Holographic Goose') 2557


In [19]:
#Find Holographic Goose num_sold in each country
nan_product = df_train[(df_train['product']=='Holographic Goose') & (df_train['num_sold'].notna())].groupby(['country'])['num_sold']
print("Mean")
print(nan_product.mean())
print("Median")
print(nan_product.median())
print("Mode")
print(nan_product.agg(pd.Series.mode))
print("Min")
print(nan_product.min())
print("Max")
print(nan_product.max())

Mean
country
Canada       241.249270
Finland      177.405944
Italy        128.415852
Kenya          5.760129
Norway       337.579194
Singapore    204.719724
Name: num_sold, dtype: float64
Median
country
Canada       234.0
Finland      188.0
Italy        136.0
Kenya          5.0
Norway       350.0
Singapore    218.0
Name: num_sold, dtype: float64
Mode
country
Canada              218.0
Finland              91.0
Italy        [66.0, 68.0]
Kenya                 5.0
Norway              171.0
Singapore           238.0
Name: num_sold, dtype: object
Min
country
Canada       200.0
Finland       65.0
Italy         44.0
Kenya          5.0
Norway       105.0
Singapore     69.0
Name: num_sold, dtype: float64
Max
country
Canada       449.0
Finland      491.0
Italy        318.0
Kenya         11.0
Norway       908.0
Singapore    469.0
Name: num_sold, dtype: float64


<ipython-input-19-7c247cb9ffdc>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  nan_product = df_train[(df_train['product']=='Holographic Goose') & (df_train['num_sold'].notna())].groupby(['country'])['num_sold']


Impute Canana NaN with 218.0 
Impute Kenya NaN with 5.0

In [20]:
len(df[df['num_sold'].isna()])

5114

In [21]:
#Impute Canada with 218
df.loc[df['country']=='Canada',['num_sold']]=df.loc[df['country']=='Canada',['num_sold']].fillna(218.0)

In [22]:
#Impute Kenya with 5
df.loc[df['country']=='Kenya',['num_sold']]=df.loc[df['country']=='Kenya',['num_sold']].fillna(5.0)

In [23]:
# nsmode = df_train['num_sold'].mode()
# df_train['num_sold']=df_train['num_sold'].fillna(nsmode[0])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 230130 entries, 2010-01-01 to 2016-12-31
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   id           230130 non-null  int64   
 1   country      230130 non-null  category
 2   store        230130 non-null  category
 3   product      230130 non-null  category
 4   num_sold     230130 non-null  float64 
 5   Day          230130 non-null  category
 6   Month        230130 non-null  category
 7   Year         230130 non-null  category
 8   Day_of_week  230130 non-null  category
dtypes: category(7), float64(1), int64(1)
memory usage: 6.8 MB


In [25]:
df_train=df.reset_index()

In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         230130 non-null  datetime64[ns]
 1   id           230130 non-null  int64         
 2   country      230130 non-null  category      
 3   store        230130 non-null  category      
 4   product      230130 non-null  category      
 5   num_sold     230130 non-null  float64       
 6   Day          230130 non-null  category      
 7   Month        230130 non-null  category      
 8   Year         230130 non-null  category      
 9   Day_of_week  230130 non-null  category      
dtypes: category(7), datetime64[ns](1), float64(1), int64(1)
memory usage: 6.8 MB


In [27]:


def objective(trial: optuna.Trial) -> float:
    #data, target = load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(df_train.drop(['id','num_sold','date'],axis=1), df_train['num_sold'], test_size=0.25)
    cats=df_train.select_dtypes('category').columns
    
    param = {
        "iterations":trial.suggest_int("iterations",1000,6000),
        "objective": trial.suggest_categorical("objective", ["MAPE"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.7, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        "eval_metric": "MAPE",
        "cat_features": cats.to_list(),
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostRegressor(**param)

    pruning_callback = CatBoostPruningCallback(trial, "MAPE")
    gbm.fit(
        train_x,
        train_y,
        eval_set=[(valid_x, valid_y)],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    mape = mean_absolute_percentage_error(valid_y, pred_labels)
    return mape


In [28]:
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="minimize"
)
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

<ipython-input-27-bfb0af462920>:27: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MAPE")
<ipython-input-27-bfb0af462920>:27: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MAPE")
<ipython-input-27-bfb0af462920>:27: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MAPE")
<ipython-input-27-bfb0af462920>:27: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MAPE")
<ipython-input-27-bfb0af462920>:27: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from

Number of finished trials: 6
Best trial:
  Value: 0.1798962889992405
  Params: 
    iterations: 3586
    objective: MAPE
    colsample_bylevel: 0.1245155129261776
    depth: 6
    boosting_type: Plain
    bootstrap_type: Bernoulli
    subsample: 0.16664035946585185


In [29]:
print(study.best_params)

{'iterations': 3586, 'objective': 'MAPE', 'colsample_bylevel': 0.1245155129261776, 'depth': 6, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.16664035946585185}


In [30]:
from catboost import Pool,cv
cats=df_train.select_dtypes('category').columns
features=df_train.drop(['id','num_sold','date'],axis=1).columns
train_pool = Pool(data=df_train.drop(['id','num_sold','date'],axis=1),
             label=df_train['num_sold'],
             cat_features=cats.to_list())

In [31]:
# from catboost import Pool,cv

#              #feature_names=features)


# # params = {"iterations": 1000,
# #           "depth": 4,
# #           "loss_function": "MAPE",
# #           "verbose": False}

# scores = cv(train_pool,
#             study.best_params,
#            fold_count=5,
#             verbose=0,
#            )

# print(scores)

In [32]:
#print(scores)

In [33]:
hyperdict=optuna.importance.get_param_importances(study)

In [34]:
print(hyperdict)

{'colsample_bylevel': 0.48038662981191355, 'iterations': 0.18749079202033672, 'bootstrap_type': 0.17493083794710917, 'boosting_type': 0.09429759100980163, 'depth': 0.0628941492108389, 'objective': 0.0}


In [35]:
#Train model with best param and iteration>1000 before prediction
model=cb.CatBoostRegressor(**study.best_params,verbose=0)
model.fit(train_pool)

In [36]:
#model.is_fitted()

In [37]:
#df_test.info()

In [38]:
predicts = model.predict(df_test.drop(['id','date'],axis=1))

In [39]:
pd.concat([df_test['id'],pd.DataFrame(predicts,columns=['num_sold'])],axis=1).reset_index(drop=True).to_csv('submission_impute_interpolate_mode3.csv',index=False)